In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import cv2
from sklearn.model_selection import train_test_split
import pickle
import os
import pandas as pd
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [4]:
%pwd

'C:\\Users\\akshi'

In [5]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [6]:
data_path='C:\\Users\\akshi\\Documents\\Verzeo\\Mask Detection'
categories=["with_mask","without_mask"]
label_dict={"with_mask":0,"without_mask":1}
labels=[0,1]


In [7]:
data=[]
target=[]
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        try: 
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(100,100))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            pass

In [8]:
print(len(data) , len(target) )

1376 1376


In [9]:
data = np.array(data)
data = data/255
data=np.reshape(data,(data.shape[0],100,100,1))
print(data.shape)

(1376, 100, 100, 1)


In [10]:
target = np.array(target)
new_target=np_utils.to_categorical(target)
print(new_target.shape)

(1376, 2)


def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img =cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img

In [11]:
def createModel() :
    model = Sequential()
    model.add(Conv2D(16,(3,3),input_shape=data.shape[1:],activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3, 3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3, 3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(50,activation="relu"))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return model

In [12]:
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.3)

In [13]:
dataGen= ImageDataGenerator(width_shift_range=0.2,   # 0.1 = 10%     IF MORE THAN 1 E.G 10 THEN IT REFFERS TO NO. OF  PIXELS EG 10 PIXELS
                            height_shift_range=0.2,
                            zoom_range=0.3,  # 0.2 MEANS CAN GO FROM 0.8 TO 1.2
                            shear_range=0.1,  # MAGNITUDE OF SHEAR ANGLE
                            rotation_range=15)  # DEGREES
dataGen.fit(train_data)

In [14]:
model=createModel()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0

In [15]:
from keras.callbacks import ModelCheckpoint
checkpoint=ModelCheckpoint("model-{epoch:03d}.model",save_best_only=True,mode="auto")
history=model.fit(train_data,train_target,epochs=30,validation_split=0.2,callbacks=[checkpoint])

Epoch 1/30
24/25 [===========================>..] - ETA: 0s - loss: 0.6615 - accuracy: 0.6068WARNING:tensorflow:From C:\Users\akshi\Anaconda3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-001.model\assets
25/25 [==============================] - 4s 149ms/step - loss: 0.6611 - accuracy: 0.6078 - val_loss: 0.6590 - val_accuracy: 0.5389
Epoch 2/30
25/25 [==============================] - 4s 146ms/step - loss: 0.5845 - accuracy: 0.6935 - val_loss: 0.5275 - val_accuracy: 0.8187
Epoch 3/30
25/25 [==============================] - 4s 151ms/step - loss: 0.4679 - accuracy: 0.7766 

In [16]:
scores = model.evaluate(test_data,test_target)
print(scores)

13/13 [==============================] - 0s 35ms/step - loss: 0.1824 - accuracy: 0.9613
[0.18237891793251038, 0.9612590670585632]


In [17]:
path_to_save= os.getcwd()
print(path_to_save)
model.save(path_to_save)

C:\Users\akshi
INFO:tensorflow:Assets written to: C:\Users\akshi\assets
